In [1]:
# import libraties
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt

In [2]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
%%timeit
total = 0
for i in np.arange(10000):
     total = i + total


3.46 ms ± 324 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
%%timeit
np.sum(np.arange(10000))

20.6 µs ± 2.31 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
df = pd.read_csv('Consumer_electronics.csv', low_memory =False,na_values = [' ','\\N'], encoding = 'utf8')

In [7]:
df.shape

(148315, 21)

In [8]:
%%timeit
Cost_price = np.zeros(len(df,))
for i in range(len(df)) : 
   Cost_price[i] = (df.iloc[i]['units'] * df.iloc[i]['product_mrp'])
df['Cost_price']=Cost_price

1min 32s ± 9.72 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
Cost_price = np.zeros(len(df,))
for i in range(len(df)) : 
   Cost_price[i] = (df.loc[i]['units'] * df.iloc[i]['product_mrp'])
df['Cost_price']=Cost_price

1min 27s ± 5.02 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
for row_index,row in df.iterrows():
    Cost_price = df['units'] * df['product_mrp']
df['Cost_price']=Cost_price

3min 57s ± 25 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit

df['Cost_price'] = df.apply(lambda x: (x.units* x.product_mrp), axis=1)

The slowest run took 4.12 times longer than the fastest. This could mean that an intermediate result is being cached.
9.35 s ± 4.15 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
df['Cost_price']= map(lambda x,y: x + y,df[["units","product_mrp"]])


5.69 ms ± 2.94 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%%timeit
df['Cost_price'] = np.vectorize(df['units'] * df['product_mrp'])

5.67 ms ± 1.98 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%%timeit
# Vectorized implementation applied on Pandas series
df['Cost_price'] = (df['units'] * df['product_mrp'])

2.11 ms ± 57.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%%timeit
# Vectorized implementation applied on NumPy arrays
df['Cost_price'] =  df['units'].values * df['product_mrp'].values

2.18 ms ± 656 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [16]:
%%timeit
scores = np.zeros(len(df,))
for i in range(len(df)):
    row = df.loc[i]
    if row['product_mrp'] < 10000:
        scores[i] = 1
    elif row['product_mrp'] >= 10000 and row['product_mrp'] <= 20000:
        scores[i] = 2
    elif row['product_mrp'] >= 20000 and row['product_mrp'] <=  30000:
        scores[i] = 3
    elif row['product_mrp'] >= 30000 and row['product_mrp'] <= 40000:
        scores[i] = 4    
    else:
        scores[i] = 5
df['Price_Range'] = scores

50.1 s ± 4.81 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
scores = np.zeros(len(df,))
mrp = df['product_mrp'].values
for i in range(len(df)):
   
    if mrp[i] < 10000:
        scores[i] = 1
    elif  mrp[i] >=10000 and  mrp[i] <= 20000:
        scores[i] = 2
    elif  mrp[i] >=20000 and  mrp[i] <= 30000:
        scores[i] = 3
    elif  mrp[i] >=30000 and  mrp[i] <= 40000:
        scores[i] = 4    
    else:
        scores[i] = 5
df['Price_Range'] = scores



167 ms ± 87.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit 
df['Price_Range'] = df['product_mrp'].apply(lambda x:'1' if x<10000  else ('2' if (x >= 10000 & x < 20000) else ('3' if (x >= 20000 & x < 30000) else ('4' if (x >= 30000 & x < 40000) else '4' ) ) ) )

64.4 ms ± 15.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%%timeit
conditions = [
    (df['product_mrp'] < 10000),
    (df['product_mrp'] > 10000) & (df['product_mrp'] < 20000),
    (df['product_mrp'] > 20000) & (df['product_mrp'] < 30000),
    (df['product_mrp'] > 30000) & (df['product_mrp'] < 40000),
    (df['product_mrp'] > 4000)
]

choices = ['1','2','3', '4','5']
df['Price_Range'] = np.select(conditions, choices, default=np.nan)

42.6 ms ± 4.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%%timeit 
df.loc[(df['product_mrp'] < 10000) , 'Price_Range'] = 1
df.loc[(df['product_mrp'] > 10000) & (df['product_mrp'] < 20000), 'Price_Range'] = 2
df.loc[(df['product_mrp'] > 20000) & (df['product_mrp'] < 30000), 'Price_Range'] = 3
df.loc[(df['product_mrp'] > 30000) & (df['product_mrp'] < 40000), 'Price_Range'] = 4
df.loc[(df['product_mrp'] > 40000) , 'Price_Range'] = 4

33.6 ms ± 4.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
%%timeit 
df['Price_Range'] = np.where(df['product_mrp'] < 10000 , 1,
                    np.where((df['product_mrp'] > 10000) & (df['product_mrp'] < 20000), 2,
                    np.where((df['product_mrp'] > 20000) & (df['product_mrp'] < 30000), 3,
                    np.where((df['product_mrp'] > 30000) & (df['product_mrp'] < 40000), 4,5))))
          

6.13 ms ± 66.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%%timeit 
name_series = pd.Series(np.random.choice(['adam', 'chang', 'eliza', 'odom'], replace=True, size=1000))

def parse_name(name):
    if name.lower().startswith('a'):
        return 'A'
    elif name.lower().startswith('e'):
        return 'E'
    elif name.lower().startswith('i'):
        return 'I'
    elif name.lower().startswith('o'):
        return 'O'
    elif name.lower().startswith('u'):
        return 'U'
    return name

parse_name_vec = np.vectorize(parse_name)
parse_name_vec(name_series)

2.36 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
%%timeit 
name_series = pd.Series(np.random.choice(['adam', 'chang', 'eliza', 'odom'], replace=True, size=1000))

def parse_name(name):
    if name.lower().startswith('a'):
        return 'A'
    elif name.lower().startswith('e'):
        return 'E'
    elif name.lower().startswith('i'):
        return 'I'
    elif name.lower().startswith('o'):
        return 'O'
    elif name.lower().startswith('u'):
        return 'U'
    return name

name_series.apply(parse_name)

2.5 ms ± 1.32 ms per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [24]:

%%timeit 

name_series = pd.Series(np.random.choice(['adam', 'chang', 'eliza', 'odom'], replace=True, size=1000))
cases = [
    name_series.str.lower().str.startswith('a'), name_series.str.lower().str.startswith('e'),
    name_series.str.lower().str.startswith('i'), name_series.str.lower().str.startswith('o'),
    name_series.str.lower().str.startswith('u')
]
replacements = 'A E I O U'.split()

np.select(cases, replacements, default=name_series)

15.7 ms ± 7.61 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
%%timeit 

df['Purchase_date'] = pd.to_datetime(df['Purchase_date']).dt.date
df['order_date'] = pd.to_datetime(df['order_date']).dt.date
df['Number_of_days'] = (df['Purchase_date'] - df['order_date']).dt.days

869 ms ± 211 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%%timeit 

df['order_date'] = pd.to_datetime(df['order_date'])
df['Purchase_date'] = pd.to_datetime(df['Purchase_date'])
def days_calc(row):
    return (row['Purchase_date']-row['order_date']).days
     
df['Number_of_days'] = df.apply(days_calc,axis=1)

8.97 s ± 2.03 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
